In [41]:
import openai
import os
from PyPDF2 import PdfReader

openai.api_key = ''

In [42]:
def get_completion(prompt, model="gpt-3.5-turbo-16k", temperature=0.1, max_tokens=1054, top_p=0.5, frequency_penalty=0.7, presence_penalty=0.7): 
    messages = [
        {"role": "system", "content": "You are a highly intelligent AI attorney assistant."},
        {"role": "user", "content": prompt}
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
    )
    return response.choices[0].message["content"]

In [43]:
def get_num_pages_from_pdf(filename):
    # creating a pdf reader object
    reader = PdfReader(filename)
    return len(reader.pages)

In [44]:
def text_from_pdf(filename, page_to_extract=-1, max_words=4097):
    # creating a pdf reader object
    reader = PdfReader(filename)
    
    text = ''
    if (page_to_extract == -1):
        for page in reader.pages:
            
            # extracting text from page
            text_tmp = page.extract_text()
            text_tmp = text_tmp.replace("\n", "")
            text += text_tmp
    else:
        if (page_to_extract > len(reader.pages)):
            return -1
        
        page = reader.pages[page_to_extract]
        # extracting text from page
        text_tmp = page.extract_text()
        text_tmp = text_tmp.replace("\n", "")
        text += text_tmp

    # Split the content into words.
    words = text.split(" ")

    # Limit the number of words.
    if len(words) > max_words:
        words = words[:max_words]

    # Return the extracted text.
    return ' '.join(words)

In [45]:
max_dim = 1024
max_sentences = range(5)
temperature = 0.1
filename = '/Users/stjames/Dropbox/Pablo/data/gpt-pdf-summarizer/text/US v Johnson-1.pdf'

num_pages = get_num_pages_from_pdf(filename)

previous_response = ""

responses_all = ""

for p in range(0, num_pages):
    text = text_from_pdf(filename, page_to_extract=p, max_words=max_dim)

    prompt = f"""Summarize the provided legal text fragment. At the end of each summary, answer the following questions in bullet point format: What is the procedural history of the case, including both criminal and immigration-related proceedings.
Were law enforcement reports  admitted during immigration proceedings? These reports could be from police, probation, and parole officers and may include documents like police reports, presentence reports, probation reports, autopsy reports, etc. If these reports were admitted in court, how much weight or importance did the court assign to them? The summaries should provide a comprehensive yet concise overview of each case, focusing particularly on the interaction between criminal and immigration proceedings and the usage of law enforcement reports. Do not repeat information summarized from the previous text fragment: Previous text framgment: ```{previous_response}``` NEW TEXT FRAGMENT TO SUMMARIZE:
        ```{text}```
        """

    response = get_completion(prompt)
    response = 'Page ' + str(p) + '\n' + response
    responses_all += response
    previous_response = response
    print ('Page ' + str(p))
responses_all = 'All Summary Fragments: ' + responses_all
print(responses_all)

Page 0
Page 1
Page 2
Page 3
Page 4
Page 5
All Summary Fragments: Page 0
Procedural History:
- Ricky Lee Johnson was found to have violated the terms of his supervised release in the United States District Court for the Eastern District of Arkansas.
- He was sentenced to 21 months' imprisonment.
- Johnson appealed the decision.

Law Enforcement Reports in Immigration Proceedings:
- It is not mentioned whether law enforcement reports were admitted during immigration proceedings.

Weight/Importance Assigned to Law Enforcement Reports:
- No information is provided regarding the weight or importance assigned to law enforcement reports in court.Page 1
Procedural History:
- In 2007, Ricky Lee Johnson was convicted of conspiracy to possess stolen mail and sentenced to fifteen months imprisonment followed by a three-year term of supervised release.
- Johnson's supervised release began on March 17, 2009.
- The probation office filed a Petition for Warrant for Offender Under Supervision, alleging

In [47]:
def get_completion2(prompt, model="gpt-3.5-turbo-16k", temperature=0.1, max_tokens=1054, top_p=0.5, frequency_penalty=0.7, presence_penalty=0.7): 
    messages = [
        {"role": "system", "content": "You are a highly intelligent AI attorney assistant."},
        {"role": "user", "content": prompt}
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
    )
    return response.choices[0].message["content"]


In [48]:
max_dim = 1024
max_sentences = range(5)
temperature = 0.1

prompt = f"""I am providing you with a set of AI generated summaries for, one for each page of a legal document. Consolidate all of these summaries into one single summary, with a maximum of 4 paragraphs. Ensure information missing at the start of the summaries is accounted for later on. The summaries should provide a comprehensive yet concise overview of each case, focusing particularly on the interaction between criminal and immigration proceedings and the usage of law enforcement reports. 

At the end of the summary, answer the following questions in bullet point format: 

What is the procedural history of the case, including both criminal and immigration-related proceedings.
Were law enforcement reports  admitted during immigration proceedings? These reports could be from police, probation, and parole officers and may include documents like police reports, pre-sentence reports, probation reports, autopsy reports, etc. 

If these reports were admitted in court, how much weight or importance did the court assign to them? TEXT: ```{responses_all}```"""

final_response = get_completion(prompt)
print(final_response)


Summary:

Ricky Lee Johnson was found to have violated the terms of his supervised release and was sentenced to 21 months' imprisonment. He appealed the decision. In 2007, Johnson was convicted of conspiracy to possess stolen mail and sentenced to fifteen months imprisonment followed by a three-year term of supervised release. During a revocation hearing, law enforcement reports, specifically a police report, were read into the record to prove allegations against Johnson. The district court found that he violated his supervised release and sentenced him accordingly. However, there is no information provided regarding whether law enforcement reports were admitted during immigration proceedings or the weight or importance assigned to them.

Procedural History:
- Ricky Lee Johnson violated his supervised release and was sentenced to 21 months' imprisonment.
- He appealed the decision.
- In 2007, Johnson was convicted of conspiracy to possess stolen mail and received a prison sentence foll